In [1]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df
import random

In [2]:
%run MODEL_LIST.ipynb            #Load the models

In [3]:
df = pd.read_csv("../data/ufc-master.csv")

In [4]:
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1
0,Alistair Overeem,Walt Harris,135.0,-155.0,135.000000,64.516129,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,2.54,-7.62,-3.0,-0.69,-0.7,-1.08,1.0,1.0
1,Claudia Gadelha,Angela Hill,-255.0,215.0,39.215686,215.000000,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,-2.54,2.54,4.0,2.36,-0.5,-2.66,1.0,1.0
2,Dan Ige,Edson Barboza,112.0,-132.0,112.000000,75.757576,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,10.16,10.16,6.0,-0.04,-0.6,-1.69,1.0,1.0
3,Krzysztof Jotko,Eryk Anders,-105.0,-105.0,95.238095,95.238095,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,0.00,-5.08,3.0,0.04,0.0,0.07,1.0,1.0
4,Song Yadong,Marlon Vera,-200.0,170.0,50.000000,170.000000,5/16/2020,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.0,0.00,7.62,5.0,-0.62,0.8,0.15,1.0,1.0


In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4000
4000
4000


In [9]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
201,Anthony Pettis,Nate Diaz,-120.0,100.0,83.333333,100.000000,2019-08-17,"Anaheim, California, USA",USA,Blue,...,4.0,5.08,10.16,-2.0,1.650000,0.200000,0.520000,0.0,1.0,1
202,Yoel Romero,Paulo Costa,-165.0,145.0,60.606061,145.000000,2019-08-17,"Anaheim, California, USA",USA,Blue,...,0.0,2.54,-2.54,14.0,4.720000,0.000000,-1.790000,0.0,1.0,1
203,Gabriel Benitez,Sodiq Yusuff,235.0,-275.0,235.000000,36.363636,2019-08-17,"Anaheim, California, USA",USA,Blue,...,-2.0,2.54,0.00,5.0,1.780000,-1.200000,-0.210000,0.0,1.0,1
204,Derek Brunson,Ian Heinisch,140.0,-160.0,140.000000,62.500000,2019-08-17,"Anaheim, California, USA",USA,Red,...,-2.0,-5.08,-12.70,4.0,-0.010000,0.000000,-1.620000,0.0,1.0,0
205,Devonte Smith,Khama Worthy,-1000.0,650.0,10.000000,650.000000,2019-08-17,"Anaheim, California, USA",USA,Blue,...,0.0,5.08,-5.08,-6.0,-1.940000,0.000000,0.000000,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4269,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4270,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4271,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Alistair Overeem,Walt Harris,135.0,-155.0,135.000000,64.516129,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,2.54,-7.62,-3.0,-0.69,-0.7,-1.08,1.0,1.0,0
1,Claudia Gadelha,Angela Hill,-255.0,215.0,39.215686,215.000000,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,-2.54,2.54,4.0,2.36,-0.5,-2.66,1.0,1.0,0
2,Dan Ige,Edson Barboza,112.0,-132.0,112.000000,75.757576,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,10.16,10.16,6.0,-0.04,-0.6,-1.69,1.0,1.0,0
3,Krzysztof Jotko,Eryk Anders,-105.0,-105.0,95.238095,95.238095,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.00,-5.08,3.0,0.04,0.0,0.07,1.0,1.0,0
4,Song Yadong,Marlon Vera,-200.0,170.0,50.000000,170.000000,2020-05-16,"Jacksonville, Florida, USA",USA,Red,...,0.0,0.00,7.62,5.0,-0.62,0.8,0.15,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Rogerio Bontorin,Raulian Paiva,100.0,-120.0,100.000000,83.333333,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,0.0,7.62,5.08,4.0,1.36,-1.1,-2.26,0.0,1.0,0
197,Geraldo de Freitas,Chris Gutierrez,-175.0,155.0,57.142857,155.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Blue,...,0.0,5.08,-12.70,-1.0,0.68,0.0,-3.62,0.0,1.0,1
198,Kazula Vargas,Alex Da Silva,240.0,-280.0,240.000000,35.714286,2019-08-10,"Montevideo, Uruguay",Uruguay,Blue,...,0.0,0.00,5.08,10.0,-0.25,0.0,2.59,0.0,1.0,1
199,Veronica Macedo,Polyana Viana,-130.0,110.0,76.923077,110.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,1.0,2.54,7.62,-4.0,1.99,0.5,0.49,0.0,1.0,0


In [10]:
def get_ev(input_df, input_model, input_features, input_labels, odds_input, min_ev = 0, verbose=False, get_total=False):
    df_sel = input_df[input_features]
    df_sel = df_sel.dropna()
    df_sel = pd.get_dummies(df_sel)
    labels_sel = input_labels[input_labels.index.isin(df_sel.index)]
    odds_sel = odds_input[odds_input.index.isin(df_sel.index)] 
    best_score = custom_cv_eval(df_sel, input_model, labels_sel, odds_sel, min_ev = min_ev, verbose=verbose, 
                                get_total=get_total)
    return best_score

In [19]:
def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
                  test_odds, verbose=True):
    model_score = 0
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    
    
    
    display(df_train.shape)
    display(labels_train.shape)
    display(odds_train.shape)
    display(df_test.shape)
    display(labels_test.shape)
    display(odds_test.shape)
    
    input_model.fit(df_train, labels_train)

    
    
    probs = input_model.predict_proba(df_test)

    
    odds_test = np.array(odds_test)    
    
    
    prepped_test = list(zip(odds_test[:, -2], odds_test[:, -1], probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    display(ev_prepped_df)
    
    #display(df_test)
    #display(df_test)
    model_score = get_ev_from_df(ev_prepped_df, print_stats = True, min_ev = input_ev, get_total=True)
    
    return(model_score)

In [25]:
m1_score = evaluate_model(model_5, features_5, min_ev_5, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m1_score)

(4000, 10)

(4000,)

(4000, 2)

(200, 10)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,135.0,-155.0,0.687804,0.312196,0
1,-255.0,215.0,0.814474,0.185526,0
2,112.0,-132.0,0.565804,0.434196,0
3,-105.0,-105.0,0.790296,0.209704,0
4,-200.0,170.0,0.733242,0.266758,0
...,...,...,...,...,...
195,100.0,-120.0,0.536701,0.463299,0
196,-175.0,155.0,0.823619,0.176381,1
197,240.0,-280.0,0.422000,0.578000,1
198,-130.0,110.0,0.858939,0.141061,0



          Number of matches: 200
          Number of bets: 196
          Number of winning bets: 128
          Number of losing bets: 68
          Number of underdog bets: 63
          Number of underdog wins: 34
          Number of underdog losses: 29
          Number of Favorite bets: 128
          Number of favorite wins: 90
          Number of favorite losses: 38
          Number of even bets: 5
          Number of even wins: 4
          Number of even losses: 1
          Profit: 2514.0446559114034
          Profit per bet: 12.826758448527569
          Profit per match: 12.570223279557018
          
          
# Matches: 200, # Bets: 196 # Wins: 128
2514.0446559114034
